**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
import sys
import os
import subprocess
import re

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags: set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = False  

# Create output directory if it doesn't exist
print(f"{os.getcwd() = }")
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Clean local only files if force_compute is set
if force_compute:
    !(cd ../../.. && make clean_local_only_files)
    # remove output directory contents
    !rm -fr output/*.* runinfo/

os.getcwd() = '/Users/terrya/Documents/ProgramData/QMCSoftware_resume/demos/talk_paper_demos/parsl_fest_2025'


rm -fr test/booktests/.ipynb_checkpoints/
chmod +x scripts/find_local_only_folders.sh > /dev/null 2>&1
for f in ; do \
		rm -f "$f"; > /dev/null 2>&1; \
	done


## 1. Sequential Execution

In [2]:
out_path = os.path.join(output_dir, "sequential_output.csv")
is_linux = sys.platform.startswith("linux")
if (not os.path.exists(out_path)) or force_compute:  # build the make command
    if is_debug:
        tests = "tb_quickstart tb_qmcpy_intro tb_lattice_random_generator"
        cmd = ["make", "booktests_no_docker", f"TESTS={tests}"]
    else:
        cmd = ["make" if not is_linux else "make -j1", "booktests_no_docker"]
    if is_linux:
        cmd = ["taskset", "-c", "0"] + cmd
    
    repo_root = _find_repo_root()

    with open(out_path, 'wb') as out_f:
        try:
            subprocess.run(cmd, cwd=repo_root, stdout=out_f, stderr=subprocess.STDOUT, check=True)
        except subprocess.CalledProcessError:
            pass

CompletedProcess(args=['make', 'booktests_no_docker'], returncode=0)

In [3]:
seq_path = os.path.join(output_dir, "sequential_output.csv")
with open(seq_path, "r") as f:
    text = f.read()
    # extract time from "Ran N tests in X.XXXs"
    match = re.search(r"Ran \d+ tests? in ([\d\.]+)s", text)

# save time to sequential_time.csv
seq_time_path = os.path.join(output_dir, "sequential_time.csv")
if match:
    sequential_time = float(match.group(1))
    with open(seq_time_path, "w") as f:
        _ = f.write(f"{sequential_time:.2f}\n")
    print(f"Sequential time: {sequential_time:.2f} seconds")
else:
    print("Warning: Could not parse sequential time from output")

# Parse individual test memory and time - pattern matches: test_name (...) ... Memory used: X.XX GB.  Test time: X.XX s
test_pattern = re.compile(r"(test_\w+)\s+\([^)]+\)\s+\.\.\.\s+Memory used:\s*([\d\.]+)\s*GB\.\s*Test time:\s*([\d\.]+)\s*s")
test_matches = test_pattern.findall(text)

# Write memory and time data to CSV files
seq_memory_path = os.path.join(output_dir, "sequential_output_memory.csv")
seq_time_detail_path = os.path.join(output_dir, "sequential_output_time.csv")

if test_matches:
    with open(seq_memory_path, "w") as f:
        _ = f.write("Notebook,Memory_GB,Time_s\n")
        for test_name, mem, time in test_matches:
            # Remove "_notebook" suffix if present
            notebook = test_name.replace("_notebook", "")
            _ = f.write(f"{notebook},{mem},{time}\n")
    
    with open(seq_time_detail_path, "w") as f:
        _ = f.write("Notebook,Memory_GB,Time_s\n")
        for test_name, mem, time in test_matches:
            # Remove "_notebook" suffix if present
            notebook = test_name.replace("_notebook", "")
            _ = f.write(f"{notebook},{mem},{time}\n")
    
    print(f"Parsed {len(test_matches)} test results")
else:
    print("Warning: Could not parse individual test results")

Sequential time: 446.12 seconds


In [4]:
# free memory
import gc
gc.collect();